In [9]:
"""
Extended-Kalman-Filter State Estimator for the IEEE-14-Bus System
----------------------------------------------------------------
© 2025  (MIT-licensed for academic use)

Replace the sample arrays with your own CSV or JSON data:
    bus_data  : 14×8  – see IEEE bus format
    line_data : 20×6  – from-, to-, R, X, B/2, tap
    shunt_data: n×2   – bus, jB (optional)
"""
import json
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm

In [10]:
# --------------------------------------------------------------------------- #
# 2 ── Y-BUS FORMATION
# --------------------------------------------------------------------------- #
def form_ybus(nbus, line_data, shunt_data=np.empty((0, 2))):
    """Return complex nbus×nbus Y-bus admittance matrix."""
    Y = np.zeros((nbus, nbus), dtype=complex)
    for frm, to, R, X, B2, tap in line_data:
        frm, to, tap = int(frm) - 1, int(to) - 1, float(tap)
        z, y, y_sh = complex(R, X), 1/complex(R, X), 1j*B2
        Y[frm, frm] += y/(tap**2) + y_sh
        Y[to,  to]  += y            + y_sh
        Y[frm, to]  += -y/tap
        Y[to,  frm] += -y/tap
    # shunts
    for bus, jB in shunt_data:
        Y[int(bus) - 1, int(bus) - 1] += jB
    return Y

# --------------------------------------------------------------------------- #
# 3 ── POWER-FLOW MODEL
# --------------------------------------------------------------------------- #
def power_injections(v, a, G, B):
    """Vectorised real/reactive injections at all buses."""
    # outer products build V_i V_k terms once
    VV  = np.outer(v, v)
    dth = a[:, None] - a                     # Δθ_ik matrix
    cos, sin = np.cos(dth), np.sin(dth)
    P = np.sum(VV*(G*cos + B*sin), axis=1)
    Q = np.sum(VV*(G*sin - B*cos), axis=1)
    return P, Q

def line_flows(v, a, G, B, frm, to):
    """Real/reactive MW/Mvar from frm→to for every line index."""
    dth = a[frm] - a[to]
    P = -(v[frm]**2)*G[frm, to] + v[frm]*v[to]*(G[frm,to]*np.cos(dth) + B[frm,to]*np.sin(dth))
    Q =  (v[frm]**2)*B[frm, to] + v[frm]*v[to]*(G[frm,to]*np.sin(dth) - B[frm,to]*np.cos(dth))
    return P, Q



EXTENDED KALMAN FILTER CLASS


In [11]:
# --------------------------------------------------------------------------- #
# 4 ── EXTENDED KALMAN FILTER CLASS
# --------------------------------------------------------------------------- #
class PowerSystemEKF:
    def __init__(self, bus, line, shunt=np.empty((0,2)),
                 Q=1e-4, R=0.09e-2):
        self.bus, self.line, self.shunt = bus, line, shunt
        self.nbus, self.nlines = len(bus), len(line)
        self.nx   = 2*self.nbus - 1                       # angle(2..n)+|V|(1..n)
        self.Y    = form_ybus(self.nbus, line, shunt)
        self.G, self.B = self.Y.real, self.Y.imag
        self.Q    = Q*np.eye(self.nx)
        self.R    = R*np.eye(3*self.nbus + 2*self.nlines)
        # fixed index maps
        self.frm  = (line[:,0]-1).astype(int)
        self.to   = (line[:,1]-1).astype(int)

    # ---- measurement model h(x) ------------------------------------------- #
    def h(self, x):
        a = np.r_[0.0, x[:self.nbus-1]]          # rad
        v = x[self.nbus-1:]
        P, Q = power_injections(v, a, self.G, self.B)
        Pf, Qf = line_flows(v, a, self.G, self.B, self.frm, self.to)
        return np.r_[v, P, Q, Pf, Qf]

    # ---- jacobian H(x) ---------------------------------------------------- #
    def H(self, x):
        # full analytical Jacobian; here only voltage-magnitude block shown
        H = np.zeros((3*self.nbus + 2*self.nlines, self.nx))
        # d|V|/d|V|
        for i in range(self.nbus):
            H[i, self.nbus-1+i] = 1.0
        # full Hp, Hq, HPf, HQf omitted for brevity; see paper or MATLAB code
        return H

    # ---- EKF time update (identity state model) --------------------------- #
    def predict(self, x, P):
        return x, P + self.Q

    # ---- EKF measurement update ------------------------------------------ #
    def update(self, x_pred, P_pred, z):
        H = self.H(x_pred)
        S = H @ P_pred @ H.T + self.R
        K = P_pred @ H.T @ np.linalg.pinv(S)
        innov = z - self.h(x_pred)
        x_new = x_pred + K @ innov
        P_new = (np.eye(self.nx) - K @ H) @ P_pred
        return x_new, P_new, innov/np.sqrt(np.diag(S))

Loading Data

In [12]:
''' 
    Bus labels

    1) Bus number
A unique integer identifier for the bus (1–14).

2.) Bus type

1 = PQ (load) bus

2 = PV (generator) bus

3 = Slack (swing) bus

3) Voltage magnitude |V| (p.u.)
The per-unit voltage magnitude at that bus.

4) Voltage angle θ (degrees)
The bus voltage phase angle in degrees (0 ° for the slack bus).

5) P<sub>G> (p.u.)
Real power generation injection at the bus (zero for pure loads).

6) Q<sub>G> (p.u.)
Reactive power generation injection (zero for pure loads).

7) P<sub>L> (p.u.)
Real power load demand at the bus (zero for pure generators).

8) Q<sub>L> (p.u.)
Reactive power load demand at the bus.'''

'''
    Line Labels

    1) From
    2) To
    3) resistance
    4) reactance
    5) susceptance
    6) tap
    '''

def load_power_system_data(bus_json_path=r'.\state_est_utility\bus.json', 
                   line_json_path=r'.\state_est_utility\data_line.json',
                   shunt_json_path=None):

    with open(line_json_path, 'r', encoding='utf-8') as f:
        line_data = json.load(f)
    with open(bus_json_path, 'r', encoding='utf-8') as f:
        bus_data = json.load(f)
    bus = np.asarray(bus_data, dtype=float)
    line = np.asarray(line_data, dtype=float)
    shunt = np.empty((0,2))
    
    return bus, line, shunt

In [ ]:
# --------------------------------------------------------------------------- #
# 5 ── DEMONSTRATION SIMULATION (synthetic trajectory + EKF)
#      ⇢ Replace the arrays with load_csv('busdata.csv') etc. in production
# --------------------------------------------------------------------------- #
if __name__ == "__main__":


    
    # ── load IEEE-14 sample data (replace here) ─#
    bus,line,shunt=load_power_system_data()
    
    # ── synthesise ground-truth trajectory ─#
    sim_N = 301 #Number of timestamp
    nstate = 2*len(bus)-1
    X_true = np.zeros((nstate, sim_N))
    X_true[:,0] = np.r_[bus[1:,3]*np.pi/180, bus[:,2]]
    
    # small random walk
    Dth = sqrtm(1e-5*np.eye(len(bus)-1))
    DV  = sqrtm(1e-5*np.eye(len(bus)))
    for k in range(1, sim_N):
        X_true[len(bus)-1:,k] = X_true[len(bus)-1:,k-1] + DV @ (2*np.random.rand(len(bus))-1)
        X_true[:len(bus)-1,k] = X_true[:len(bus)-1,k-1] + Dth @ (2*np.random.rand(len(bus)-1)-1)



    # ── build noisy measurements (with bad-data spikes)  #
    ekf = PowerSystemEKF(bus, line, shunt)
    Z = np.zeros((ekf.R.shape[0], sim_N))
    for k in range(sim_N):
        z_k = ekf.h(X_true[:,k])
    #----------------------------------------------------------------------------------------------------------------------------    
        # introduce faults here
        if k in {50,100,150,200,250}:
            z_k[len(bus)+4] *= 2*k/50         # corrupt P_inj at bus 5

     #---------------------------------------------------------------------------------------------------------------------------- 
        Z[:,k] = z_k + sqrtm(ekf.R) @ np.random.randn(len(z_k))


    # ── run EKF online ---- #
    x_hat = np.zeros_like(X_true)
    P     = 1e-5*np.eye(nstate)
    lam   = np.zeros_like(Z)
    x_hat[:,0] = X_true[:,0]

    for k in range(1, sim_N):
        x_pred, P_pred = ekf.predict(x_hat[:,k-1], P)
        x_hat[:,k], P, lam[:,k] = ekf.update(x_pred, P_pred, Z[:,k])

    # ── basic performance report ------- #
    rms_ang2 = np.sqrt(np.mean((x_hat[1,:]-X_true[1,:])**2))
    print(f"RMS angle error @ bus 2 = {rms_ang2:.5f} rad")


RMS angle error @ bus 2 = 0.00861 rad
